In [28]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.simplefilter('ignore')
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

Load the tips dataset.



In [3]:
df = sns.load_dataset('tips')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 244 entries, 0 to 243
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   total_bill  244 non-null    float64 
 1   tip         244 non-null    float64 
 2   sex         244 non-null    category
 3   smoker      244 non-null    category
 4   day         244 non-null    category
 5   time        244 non-null    category
 6   size        244 non-null    int64   
dtypes: category(4), float64(2), int64(1)
memory usage: 7.4 KB


# Create a column named price_per_person. This should be the total bill divided by the party size.


In [21]:
df['price_per_person']= df.total_bill / df['size']
df.sex = df.sex.apply(lambda x: 1 if x == 'Male' else 0)
df.time = df.time.apply(lambda x: 1 if x == 'Dinner' else 0)
df.smoker = df.smoker.apply(lambda x: 1 if x == 'Yes' else 0)
dummy_df = pd.get_dummies(df['day'])
xtrain = df[['total_bill','size','price_per_person']]
ytrain = df[['tip']]
dummy_df

,Thur,Fri,Sat,Sun
0,0,0,0,1
1,0,0,0,1
2,0,0,0,1
3,0,0,0,1
4,0,0,0,1
...,...,...,...,...
239,0,0,1,0
240,0,0,1,0
241,0,0,1,0
242,0,0,1,0


# Before using any of the methods discussed in the lesson, which features do you think would be most important for predicting the tip amount?
From my examination, the best features with which to predict the tip amount will be total_bill and price_per_person

# Use select k best to select the top 2 features for predicting tip amount. What are they?


In [36]:
from sklearn.feature_selection import SelectKBest,f_regression,RFE
from sklearn.linear_model import LinearRegression
skb = SelectKBest(f_regression,k=2)
skb.fit(xtrain,ytrain)
feature_mask = skb.get_support()
f_feature = xtrain.iloc[:,feature_mask]
f_feature.columns.tolist()

['total_bill', 'size']

Use recursive feature elimination to select the top 2 features for tip amount. What are they?


In [34]:
linreg = LinearRegression()
rfe = RFE(linreg,n_features_to_select=2)
rfe.fit(xtrain,ytrain)
feature_mask = rfe.support_
xtrain.iloc[:,feature_mask]

,total_bill,price_per_person
0,16.99,8.495000
1,10.34,3.446667
2,21.01,7.003333
3,23.68,11.840000
4,24.59,6.147500
...,...,...
239,29.03,9.676667
240,27.18,13.590000
241,22.67,11.335000
242,17.82,8.910000


Why do you think select k best and recursive feature elimination might give different answers for the top features? Does this change as you change the number of features you are selecting?


K Best Features doesn't actually do the linear regression test, so it is not able to predict results in the way that RFE does.

Write a function named select_kbest that takes in the predictors (X), the target (y), and the number of features to select (k) and returns the names of the top k selected features based on the SelectKBest class. Test your function with the tips dataset. You should see the same results as when you 
did the process manually.


In [38]:
def select_kbest(xtrain,ytrain,k):
    kbest = SelectKBest(f_regression,k=k)
    kbest.fit(xtrain,ytrain)
    feature_mask = kbest.get_support()
    return xtrain.iloc[:,feature_mask].columns.tolist()
select_kbest(xtrain,ytrain,2)

['total_bill', 'size']


Write a function named rfe that takes in the predictors, the target, and the number of features to select. It should return the top k features based on the RFE class. Test your function with the tips dataset. You should see the same results as when you did the process manually.


In [40]:
def rfe(xtrain, ytrain, k):
    linreg = LinearRegression()
    rf = RFE(linreg,n_features_to_select=k)
    rf.fit(xtrain,ytrain)
    return xtrain.iloc[:,rf.support_].columns.tolist()
rfe(xtrain,ytrain,2)

['total_bill', 'price_per_person']


Load the swiss dataset and use all the other features to predict Fertility. Find the top 3 features using both select k best and recursive feature elimination (use the functions you just built to help you out).

In [45]:
from pydataset import data
swiss = data('swiss')
xswiss = swiss[['Agriculture','Examination','Education','Catholic','Infant.Mortality']]
yswiss = swiss[['Fertility']]
kb = select_kbest(xswiss,yswiss,3)
rf = rfe(xswiss,yswiss,3)
pd.DataFrame({'K Best':kb,'Recursive':rf})


,K Best,Recursive
0,Examination,Examination
1,Education,Education
2,Catholic,Infant.Mortality
